In [ ]:
import pandas as pd
import numpy as np
import os
print(os.getcwd())

In [ ]:
os.chdir("../..")

In [ ]:
%load_ext autoreload
%autoreload 2
from countsdiff.data.process_scrna import SingleCellDataset
from countsdiff.utils.metrics import scFID
from baselines.MAGIC.magic_wrapper import MAGICWrapper

In [ ]:
trainset = SingleCellDataset("data/dnadiff/filtered_hca_data.hdf5", "train", ["cell_type", "Batch", "Development_day", "disease", "sex"])
testset = SingleCellDataset("data/dnadiff/filtered_hca_data.hdf5", "test", ["cell_type", "Batch", "Development_day", "disease", "sex"])

In [ ]:
obs_dict = testset.get_obs_dict(unique=True)

In [ ]:
model_filename = "data/dnadiff/2024-02-12-scvi-homo-sapiens/scvi.model"
scfid = scFID(gene_names = testset.gene_names, categorical_covariates= obs_dict, feature_model_path=model_filename)

In [ ]:
mwrapper = MAGICWrapper(knn=35, n_jobs = -1)

In [ ]:
impute_mask = np.random.rand(*testset.counts.shape) < 0.4

In [ ]:
train_mask = np.ones(shape = trainset.counts.shape, dtype = bool)

In [ ]:
valid_mask = ~testset.missingness_mask.numpy()

In [ ]:
impute_mask = impute_mask*valid_mask

In [ ]:
total_counts = np.concatenate((trainset.counts.numpy(), testset.counts.numpy()), axis = 0)

In [ ]:
total_counts

In [ ]:
total_imp_mask = np.concatenate((~train_mask, impute_mask))

In [ ]:
total_imp_mask

In [ ]:
total_valid_mask = np.concatenate((train_mask, valid_mask))

In [ ]:
imputed_data = mwrapper.impute_data(total_counts, total_imp_mask, gene_names = testset.gene_names, valid_mask = total_valid_mask)